# Matching algorithm via keywords

### Prerequisites

Download processed datasets via `git lfs fetch --include "data/processed/*.csv"`

or create them from scratch with [preprocess.ipynb](./preprocess.ipynb)

In [1]:
!pip install pandas
!pip install scikit-learn
!pip install nltk
!pip install transformers
!pip install numpy

!pip install rake-nltk
!pip install pip install git+https://github.com/LIAAD/yake
!pip install keybert

  Cloning https://github.com/LIAAD/yake to /private/var/folders/3y/lpnlj_f962vbckl7626fst540000gq/T/pip-req-build-l8z6f3hc
  Running command git clone --filter=blob:none --quiet https://github.com/LIAAD/yake /private/var/folders/3y/lpnlj_f962vbckl7626fst540000gq/T/pip-req-build-l8z6f3hc
  Resolved https://github.com/LIAAD/yake to commit 374fc1c1c19eb080d5b6115cbb8d4a4324392e54
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [20]:
import os
import os.path as path
import pandas as pd
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np

from rake_nltk import Rake
import yake
from keybert import KeyBERT

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alv.popov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/alv.popov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
root_dir = path.dirname(path.abspath(os.getcwd()))
data_dir = path.join(root_dir, 'data')
processed_data_dir = path.join(data_dir, 'processed')

processed_vacancies_path = path.join(processed_data_dir, 'vacancies.csv')
processed_cvs_path = path.join(processed_data_dir, 'cvs.csv')

processed_cvs_path

'/Users/alv.popov/prj/DL-CASE2/data/processed/cvs.csv'

In [4]:
processed_vacancies = pd.read_csv(processed_vacancies_path)
processed_vacancies = processed_vacancies[processed_vacancies['text'].notnull()]
processed_vacancies

,text
0,Project R’n’D / Технолог R’n’D . MF Kitchen — ...
1,"Продавец-кассир (Москва, Богданова, 16) . Обяз..."
2,"Продавец-кассир (Москва, Новогиреевская, 11/36..."
3,"Product owner бизнес–стрима ""Аналитика и Отчет..."
4,Business Development Director (CTO) . Студия О...
...,...
96220,"Оператор 1C . ТД «ЛФБ» - семейная компания, ..."
96221,Стажер Управления финансирования торговли . КР...
96222,Преподаватель математики/информатики . Обязанн...
96223,Программист С++ / Linux . Требования: Знание ...


In [5]:
processed_cvs = pd.read_csv(processed_cvs_path)
processed_cvs = processed_cvs[processed_cvs['text'].notnull()]
processed_cvs

,text
0,"Мужчина , 39 лет , родился 27 ноября 1979 . С..."
1,"Мужчина , 60 лет , родился 20 марта 1959 . Те..."
2,"Женщина , 36 лет , родилась 12 августа 1982 ...."
3,"Мужчина , 38 лет , родился 25 июня 1980 . Веб..."
4,"Женщина , 26 лет , родилась 3 марта 1993 . Ре..."
...,...
44739,"Мужчина , 30 лет , родился 17 января 1989 . Ф..."
44740,"Мужчина , 27 лет , родился 5 марта 1992 . Сис..."
44741,"Женщина , 48 лет , родилась 26 декабря 1970 ...."
44742,"Мужчина , 24 года , родился 6 октября 1994 . ..."


In [15]:
# def extract_keywords(text, max_length=5, n_keywords=10):
#     rake = Rake(language='russian', max_length=max_length)
#     rake.extract_keywords_from_text(str(text))
#     return rake.get_ranked_phrases()[:n_keywords]

rake = Rake(language='russian')

def extract_keywords(text, max_length=5, n_keywords=10):
    extractor = yake.KeywordExtractor(
        stopwords=rake.to_ignore,
        windowsSize=max_length,
        top=n_keywords
    )
    scored_keywords = extractor.extract_keywords(str(text))
    return list(map(lambda x: x[0], scored_keywords))


# !!!! KeyBert не разбивает на токены,  !!!!

# extractor = KeyBERT('cointegrated/LaBSE-en-ru')

# def extract_keywords(text, max_length=5, n_keywords=10):
#     scored_keywords = extractor.extract_keywords(
#         str(text),
#         keyphrase_ngram_range=(1, max_length),
#         stop_words=rake.to_ignore,
#         top_n=n_keywords,
#     )
#     return list(map(lambda x: x[0], scored_keywords))

In [16]:
vacancy = processed_vacancies['text'].iloc[121]
vacancy_keywords = extract_keywords(vacancy, max_length=1, n_keywords=30)

display(vacancy)
display(vacancy_keywords)

'C++ разработчик в Германию . Мы ищем C++ разработчиков разных уровней от junior до senior с переездом в Берлин.  Наш клиент - немецкая компания, 17 лет на рынке. Их разработка - программное обеспечение для создания диаграмм и макетов, распознавания их с бумажных носителей. Продукт интегрируется в PowerPoint и автоматизирует его работу за счет применения сложных алгоритмов. Повышает эффективность и качество создания слайдов, в несколько раз экономит время пользователю при подготовке презентации. Поставляется с внутренней таблицей данных на основе Excel и интеллектуальными ссылками. Продукт работает и по Windows, и под Mac. При его выходе на рынок это была настоящая революция. Компания занимала первые места в рейтингах лучших стартапов Германии и быстрорастущих бизнесов. У компании 700 000 клиентов по всему миру. Среди них, например, Deloitte, Ernst & Young, Coca-Cola, DHL, 3M, Cisco. В чем еще участвует компания:  спонсор Standard C++ Foundation - ведущего мирового некоммерческого сооб

['разработчик в Германию',
 'разработчиков разных уровней',
 'разных уровней',
 'уровней от junior',
 'junior до senior',
 'senior с переездом',
 'переездом в Берлин',
 'разработчиков разных',
 'Германию',
 'Наш клиент',
 'лет на рынке',
 'разработчик',
 'Продукт',
 'английском языке',
 'ищем',
 'разных',
 'уровней',
 'junior',
 'senior',
 'переездом',
 'компания',
 'Берлин',
 'немецкая компания',
 'качество создания слайдов',
 'Германии',
 'CTO',
 'Джоном Форрестом',
 'создания диаграмм',
 'создания',
 'применения сложных алгоритмов']

In [17]:
cv = processed_cvs['text'].iloc[815]
cv_keywords = extract_keywords(cv, n_keywords=30)

display(cv)
display(cv_keywords)

'Женщина ,  24 года , родилась 31 октября 1994 . It-специалист, менеджер . Белгород , не готова к переезду , не готова к командировкам . частичная занятость, полная занятость . полный день, удаленная работа . Опыт работы 8 месяцев  It-специалист, менеджер 25 000 руб. Информационные технологии, интернет, телеком Сетевые технологии Телекоммуникации Занятость: частичная занятость, полная занятость График работы: полный день, удаленная работа Опыт работы 8 месяцев Апрель 2018 — Ноябрь  2018 8 месяцев ТЭКБ Менеджер по продажам Грамотная речь, умение работать с клиентами) Ключевые навыки Теперь резюме открыто всему интернету — изменить можно в настройках видимости. Коммуникабельная Общительная Исполнительная Начатое довожу до конца Возникли неполадки. Попробуйте еще раз. Опыт вождения Имеется собственный автомобиль Права категории B Обо мне Грамотная речь, презентабельная внешность , коммуникабельность, ответственность‚ исполнительность, многозадачность, стрессоустойчивость, обучаемость, вни

['месяцев ТЭКБ Менеджер',
 'ТЭКБ Менеджер',
 'продажам Грамотная речь',
 'месяцев It-специалист',
 'полная занятость График',
 'месяцев ТЭКБ',
 'менеджер',
 'Грамотная речь',
 'Менеджер по продажам',
 'полный день',
 'занятость График работы',
 'It-специалист',
 'удаленная работа Опыт',
 'частичная занятость',
 'продажам Грамотная',
 'готова к переезду',
 'готова к командировкам',
 'Общительная Исполнительная Начатое',
 'полная занятость',
 'Сетевые технологии Телекоммуникации',
 'месяцев Апрель',
 'Опыт работы',
 'удаленная работа',
 'работа Опыт работы',
 'месяцев',
 'технологии Телекоммуникации Занятость',
 'ТЭКБ',
 'занятость',
 'Белгород',
 'умение работать']

In [21]:
model = SentenceTransformer('cointegrated/LaBSE-en-ru')

res = cosine_similarity(model.encode(vacancy_keywords), model.encode(cv_keywords)).ravel()
print(
f"""
Min: {res.min()}
Mean: {res.mean()}
Max: {res.max()}
Harmonic: {len(res) / sum(1 / res)}
Median: {np.median(res)}
"""
)


Min: -0.005935093853622675
Mean: 0.27975282073020935
Max: 0.7031041979789734
Harmonic: 0.3452948097313967
Median: 0.27299636602401733



In [22]:
def calc_metric(v1, v2):
    return np.median(cosine_similarity(v1, v2))

calc_metric(model.encode(vacancy_keywords), model.encode(cv_keywords))

0.27299637

In [23]:
n_items = 100
n_keywords = 10
max_length = 5

def encode_batch(keywords_batch):
    encoded = model.encode(np.concatenate(keywords_batch).tolist())
    lengths = np.fromiter(map(len, keywords_batch), dtype=int)
    encoded_keywords_iter = (encoded[end - l: end] for end, l in zip(np.cumsum(lengths), lengths))
    return np.fromiter(encoded_keywords_iter, dtype=np.ndarray)

def get_encodings(dataset):
    dataset = dataset.apply(lambda x: extract_keywords(x, max_length, n_keywords))
    dataset = dataset.tolist()
    return encode_batch(dataset)

sampled_vacancies = processed_vacancies.sample(n_items)
sampled_cvs = processed_cvs.sample(n_items)

vacancies_encodings = get_encodings(sampled_vacancies['text'])
cvs_encodings = get_encodings(sampled_cvs['text'])

results = np.array([[calc_metric(cv_encoding, vac_encoding) for cv_encoding in cvs_encodings] for vac_encoding in vacancies_encodings])
print('\n'.join(map(lambda x: '\t'.join(map(str, x)), results[:10, :10])))

0.27295914	0.30121636	0.25362828	0.24358062	0.29201055	0.22992617	0.25713181	0.30777884	0.25362903	0.27860218
0.21230206	0.23829053	0.22328493	0.27419066	0.25597212	0.1892893	0.21603414	0.29591143	0.22867414	0.22170061
0.2938247	0.26444995	0.19210538	0.2492396	0.24684174	0.20161954	0.24709846	0.29841295	0.2500275	0.2449508
0.23629296	0.24705368	0.23591316	0.24027659	0.25954872	0.19471578	0.23043692	0.3013603	0.19969326	0.22657508
0.24648574	0.2685027	0.23113482	0.28407404	0.25395507	0.22926432	0.30904704	0.3361689	0.26167464	0.24531856
0.18539181	0.2502637	0.1921595	0.17520857	0.23550572	0.23626813	0.19599904	0.24755815	0.14950567	0.22162858
0.32844064	0.30562222	0.28722262	0.2343201	0.3223598	0.4542704	0.2591659	0.3286007	0.2002295	0.3041939
0.22065616	0.21352707	0.2048118	0.21548031	0.23264842	0.18847555	0.2864035	0.25637072	0.22647119	0.2439503
0.24280414	0.23901516	0.23098895	0.26828468	0.23467708	0.22539166	0.27083695	0.2845863	0.24803486	0.21259461
0.2760768	0.24408865	0.21810636

In [27]:
np.where(results > 0.45)

(array([6]), array([5]))

In [25]:
kw1 = extract_keywords(sampled_vacancies['text'].iloc[6], max_length, n_keywords)
kw2 = extract_keywords(sampled_cvs['text'].iloc[5], max_length, n_keywords)

metric = calc_metric(model.encode(kw1), model.encode(kw2))
display(kw1, kw2, metric)

['Требуется инженер проекта',
 'инженер проекта Инженерные',
 'проекта Инженерные системы',
 'главный инженер проекта',
 'Инженер проекта',
 'Требуется инженер',
 'проекта Инженерные',
 'Инженерные системы',
 'ведущий инженер',
 'главный инженер']

['Инженер ЭВТ Системное',
 'ЭВТ Системное администрирование',
 'Ведущий специалист-эксперт Системное',
 'лицей-интернат Инженер ЭВТ',
 'Республиканский лицей-интернат Инженер',
 'Системное администрирование',
 'Инженер ЭВТ',
 'Ижевска Ведущий специалист-эксперт',
 'Администрация города Ижевска',
 'города Ижевска Ведущий']

0.45427048